# 1.2 Exercise 
1. For a perceptron, the positive part is always 1 and negetive part is always 0, times a positive number could nor change their sign.
2. $w \times x+b \longrightarrow c (x \times x+b)\longrightarrow\infty$

# 1.4 Execcise
1. I think its same with 10digits to 2digits.

# 1.5
The gradient descent algorithm


 $C(w,b)\equiv\frac{1}{2n}\sum_x\|y(x)-a\|^2$


$ \Delta C\approx\frac{\partial C}{\partial\nu_1}\Delta\nu_1+\frac{\partial C}{\partial\nu_2}\Delta\nu_2$  

$\Delta C\approx\nabla C\cdot\Delta\nu$

$\Delta\nu=-\eta\nabla C$


$\nu\to\nu^{\prime}=\nu-\eta\nabla C$
# Exercise
1. Cauchy-Schwarz Inequality $|\langle\mathbf{u},\mathbf{v}\rangle|\leq\|\mathbf{u}\|\cdot\|\mathbf{v}\|$

I think it like a Inverse function.

2. Quadratic Function


# stochastic gradient descent
Chose a mini-batch to calculate $C $


$\frac{\sum_{j=1}^m\nabla C_{X_j}}{m}\approx\frac{\sum_x\nabla C_x}{n}=\nabla C$


 $w_k\quad\to\quad w_k^{\prime}=w_k-\frac{\eta}{m}\sum_j\frac{\partial C_{X_j}}{\partial w_k}$
 
 
 $b_l\quad\to\quad b_l^{\prime}=b_l-\frac{\eta}{m}\sum_j\frac{\partial C_{X_j}}{\partial b_l},$


  # Exercise 
   Fast but inaccurate

In [7]:
import numpy as np
import random

class NeuralNetwork:
    """
    全连接前馈神经网络实现
    支持反向传播和随机梯度下降训练
    """
    
    def __init__(self, layer_sizes):
        """
        初始化神经网络
        
        参数:
        layer_sizes -- 包含各层神经元数量的列表，如[784, 30, 10]表示:
                       - 输入层784个神经元
                       - 隐藏层30个神经元
                       - 输出层10个神经元
        """
        self.num_layers = len(layer_sizes)
        self.layer_sizes = layer_sizes
        
        # 初始化偏置向量(每层一个向量，从第二层开始)
        # 使用标准正态分布随机初始化
        self.biases = [np.random.randn(y, 1) for y in layer_sizes[1:]]
        
        # 初始化权重矩阵(每层之间一个矩阵)
        # 使用标准正态分布随机初始化
        self.weights = [np.random.randn(y, x) 
                       for x, y in zip(layer_sizes[:-1], layer_sizes[1:])]
    
    def feedforward(self, a):
        """
        前向传播计算网络输出
        
        参数:
        a -- 输入向量(二维numpy数组)
        
        返回:
        网络的输出
        """
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a
    
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """
        使用随机梯度下降训练网络
        
        参数:
        training_data -- 训练数据集，格式为[(x, y), ...]，其中:
                        x是输入向量，y是期望输出向量
        epochs -- 训练轮数
        mini_batch_size -- 小批量大小
        eta -- 学习率
        test_data -- 测试数据集(可选)，用于评估训练效果
        """
        n = len(training_data)
        if test_data:
            n_test = len(test_data)
        
        for epoch in range(epochs):
            # 每轮训练前打乱数据顺序
            random.shuffle(training_data)
            
            # 将训练数据分成多个小批量
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)
            ]
            
            # 用每个小批量更新网络参数
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            
            # 输出训练进度
            if test_data:
                accuracy = self.evaluate(test_data) / n_test
                print(f"Epoch {epoch}: Accuracy {accuracy:.2%}")
            else:
                print(f"Epoch {epoch} complete")
    
    def update_mini_batch(self, mini_batch, eta):
        """
        用一个小批量数据更新网络参数
        
        参数:
        mini_batch -- 小批量数据，格式为[(x, y), ...]
        eta -- 学习率
        """
        # 初始化梯度累加器
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # 计算小批量中所有样本的梯度总和
        for x, y in mini_batch:
            # 对每个样本计算梯度
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            # 累加梯度
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        # 更新参数: 参数 = 参数 - (学习率/批量大小) * 梯度
        self.weights = [w - (eta/len(mini_batch)) * nw
                       for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch)) * nb
                      for b, nb in zip(self.biases, nabla_b)]
    
    def backprop(self, x, y):
        """
        反向传播算法
        
        参数:
        x -- 输入样本
        y -- 期望输出
        
        返回:
        包含梯度值的元组 (nabla_b, nabla_w)，格式与self.biases和self.weights相同
        """
        # 初始化梯度
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # 前向传播(保存中间值)
        activation = x  # 当前层激活值
        activations = [x]  # 存储各层激活值
        zs = []  # 存储各层加权输入(z = w*a + b)
        
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # 反向传播
        # 输出层误差
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # 从倒数第二层开始反向传播误差
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        
        return (nabla_b, nabla_w)
    
    def evaluate(self, test_data):
        """
        评估网络在测试数据上的表现
        
        参数:
        test_data -- 测试数据集，格式为[(x, y), ...]
        
        返回:
        正确分类的数量
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                       for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    
    def cost_derivative(self, output_activations, y):
        """
        二次代价函数的导数
        
        参数:
        output_activations -- 网络输出
        y -- 期望输出
        
        返回:
        输出误差
        """
        return (output_activations - y)


# 辅助函数
def sigmoid(z):
    """S型激活函数"""
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    """S型函数的导数"""
    return sigmoid(z) * (1 - sigmoid(z))


In [5]:
import mnist_loader
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

UnicodeDecodeError: 'ascii' codec can't decode byte 0x90 in position 614: ordinal not in range(128)